#### Imports

In [ ]:
import numpy as np
import cv2
import skimage
from skimage import morphology
from scipy import ndimage
import os,glob
from os import listdir,makedirs
from os.path import isfile,join

#### Otsu segmentation

In [ ]:
path = 'C:/Users/User1/Downloads/Mango/Anthracnose' # Source Folder
dstpath = 'C:/Users/User1/Downloads/MangoOtsu/AnthracnoseOtsu' # Destination Folder
try:
    makedirs(dstpath)
except:
    print ("Directory already exist, images will be written in same folder")
# Folder won't used
files = list(filter(lambda f: isfile(join(path,f)), listdir(path)))
for image in files:
    try:
        inputImage = cv2.imread(os.path.join(path,image))
        # Convert RGB to grayscale:
        grayscaleImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)

        Gaussian = cv2.GaussianBlur(grayscaleImage, (7, 7), 0) 

        # (Otsu) Threshold:
        thresh, binaryImage = cv2.threshold(Gaussian, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        #Inverting the mask
        binaryImage = cv2.bitwise_not(binaryImage) 

        #Filling holes, closing and eroding operations
        bitImage=ndimage.binary_fill_holes(binaryImage,structure=np.ones((3,3))).astype(np.uint8)
        bitImage=skimage.morphology.binary_closing(bitImage).astype(np.uint8)
        bitImage=skimage.morphology.erosion(bitImage).astype(np.uint8)

        #Integrating mask with image
        result = cv2.bitwise_and(inputImage, inputImage,mask=bitImage)

        dstPath = join(dstpath,image)
        cv2.imwrite(dstPath,result)
    except:
        print ("{} is not converted".format(image))

#### HSV based segmentation

In [ ]:
path = 'C:/Users/User1/Downloads/Mango/PowderyMildew' # Source Folder
dstpath = 'C:/Users/User1/Downloads/Mangohsv/PowderyMildewhsv' # Destination Folder
try:
    makedirs(dstpath)
except:
    print ("Directory already exist, images will be written in same folder")
# Folder won't used
files = list(filter(lambda f: isfile(join(path,f)), listdir(path)))
for image in files:
    try:
        inputImage = cv2.imread(os.path.join(path,image))
        # create hsv
        hsv = cv2.cvtColor(inputImage, cv2.COLOR_BGR2HSV)
         # set lower and upper color limits
        low_val = (0,60,0)
        high_val = (179,255,255)
        # Threshold the HSV image 
        mask = cv2.inRange(hsv, low_val,high_val)
        # remove noise
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel=np.ones((8,8),dtype=np.uint8))

        grayscaleImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)
        # apply mask to original image
        result = cv2.bitwise_and(grayscaleImage, grayscaleImage,mask=mask)
        
        dstPath = join(dstpath,image)
        cv2.imwrite(dstPath,result)
    except:
        print ("{} is not converted".format(image))